<a href="https://colab.research.google.com/github/Anna-Zaitseva/HH_vacancies_analysis/blob/master/Project_2_%D0%9D%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA_%D1%88%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [ ]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import pprint
import collections

In [ ]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# текст запроса
query_3_1 = f'''SELECT count(id)
                FROM public.vacancies
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
print('Количество вакансий в базе:')
df['count'][0]

Количество вакансий в базе:


49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса
query_3_2 = f'''SELECT count(id)
                FROM public.employers
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
print('Количество работодателей в базе:')
df['count'][0]

Количество работодателей в базе:


23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
query_3_3 = f'''SELECT count(id)
                FROM public.areas
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
print('Количество регионов в открытыми вакансиями:')
df['count'][0]

Количество регионов в открытыми вакансиями:


1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
query_3_4 = f'''SELECT count(id)
                FROM public.industries
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
print('Количество сфер деятельности:')
df['count'][0]

Количество сфер деятельности:


294

***

<center>Выводы по предварительному анализу.</center>

Выборка очень большая, чуть более 49 тысяч вакансий в 294 видах деятельности. В среднем на каждого работодателя приходится по 2 вакансии в 1362 регионах (населенных пунктах).

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
query_4_1 = f'''SELECT 
                      A.name as area, 
                      count(V.name) as cnt
                FROM public.areas as A
                JOIN public.vacancies as V on A.id=V.area_id
                GROUP BY A.name
                ORDER BY count(V.name) desc
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
display(df)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2 = f'''SELECT count(V.name) as "Количество вакансий"
                FROM public.vacancies as V
                WHERE salary_from is not NULL or salary_to is not NULL
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
display(df)

,Количество вакансий
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
query_4_3 = f'''SELECT 
                      round(avg(V.salary_from),0) as "Нижняя граница", 
                      round(avg(V.salary_to),0) as "Верхняя граница"
                FROM public.vacancies as V
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
display(df)

,Нижняя граница,Верхняя граница
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = f'''SELECT 
                      V.schedule as "Тип рабочего графика", 
                      V.employment as "Тип трудоустройства", 
                      count(V.id) as "Количество вакансий"
                FROM public.vacancies as V
                GROUP BY V.schedule, V.employment
                ORDER BY count(V.id) desc
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
display(df)

,Тип рабочего графика,Тип трудоустройства,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
query_4_5 = f'''SELECT 
                      V.experience as "Требуемый опыт работы", 
                      count(V.id) as "Количество вакансий"
                FROM public.vacancies as V
                GROUP BY V.experience
                ORDER BY count(V.id) asc
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
display(df)

,Требуемый опыт работы,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


<center>Выводы по детальному анализу вакансий.</center>

Как видно географический охват выборки достаточно большой несмотря на то, что hh.ru российский сервис. В лидерах по количеству вакансий (топ 5) не только крупнейшие российские города (Москва, Санкт-Петербург и Новосибирск), но и столица Белоруссии Минск и бывшая столица Казахстана Алматы. А на первых 100 позициях можно увидеть вакансии как из крупных городов России (более 500 тыс.населения), так и из соседних стран. Например, Азербайджан (Баку), Грузия (Тбилиси, Батуми) и даже Кипр. 

Половина вакансий опубликованы без указания заработной платы. Работодатель намеренно скрывает возможные цифры по следующим причинам: 

1.   Заработная плата существенно выше средней по рынку, рекрутер не успевает просматривать входящие резюме.
2.   Заработная плата существенно ниже средней по рынку, и рекрутер при встрече будет делать упор на другие положительные стороны компании.
3.   Реальной вакансии нет, но есть оплаченный доступ к порталу. Рекрутер просматривает входящие резюме, чтобы иметь представления о круге обязанностей и средней ожидаемой заработной плате. Возможно, собирает контакты интересных кандидатов.

Указанная в вакансии вилка зарплаты, говорит о том, что компания готова платить указанный минимум. Но чтобы получить максимум, кандидат, помимо профессиональных знаний, должен проявить навыки переговорщика.

Более 70 % (около 35 тыс.) выборки - вакансии на полный день и полную занятость. Далее с большим отрывом следуют удаленная работа и гибкий график ***на полной занятости***: 16% и 3%. Такой спрос на полную занятость со стороны работодателей объясняется прежде всего объемами и спецификой работ. Когда появляется работа, которую можно сделать за полдня, логичнее распределить ее между действующими сотрудниками (отдать конкретному сотруднику или аутсорсинг) и добавить к заработной плате. Нового сотрудника придется вводить в курс дела и обучать, что требует времени. Возможно, человек не пройдет испытательный срок, поиск и обучение нужно будет начинать сначала. Работодатель старается минимизировать риски и не терять ресурсы в виде времени и упущенной прибыли. Когда для выполнения работы требуется полный рабочий день, тогда оптимальным решением будет взять нового человека, т.к. качество работы человека, работающего за двоих, снизится. 

На четвертом месте удаленная работа с частичной занятостью (около 2,5%) и на пятом сменный график с полной занятостью (около 1,9%). Далее преобладают графики с временной занятостью: стажировки, проектные работы, вахтовый метод.

Проще всего найти работу человеку, у которого опыт работы от 1 года до 3 лет (около 52% предложений). Он уже не новичок, но понимает, что нужно подтягивать знания и нарабатывать опыт, соответственно требует меньше. Обычно таких соискателей ищут на рутинные и тривиальные задачи. 29% вакасий для более опытных соискателей - от 3 до 6 лет. Обычно после 3 лет работы появляются специфические профессиональные навыки, а также способность решать нетривиальные задачи, амбициозные сотрудники метят на руководящие должности. Труднее всего новичкам без опыта: количество вакансий для них 14,5%, так как не каждый работодатель готов обучать с нуля. Соискателям с опытом от 6 лет легче: количество вакансий для этой категории примерно 4,5% (обычно это руководящие должности), но они могут пойти на понижение: выполнять работу, где требуется меньше опыта и навыков.



***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1 = f'''SELECT 
                      E.name as "Наименование работодателя", 
                      count(V.id) as "Количество вакансий"
                FROM public.employers as E
                JOIN public.vacancies as V on E.id=V.employer_id
                GROUP BY E.name
                ORDER BY count(V.id) desc
                LIMIT 5
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
display(df)

,Наименование работодателя,Количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f'''SELECT 
                      A.name as "Регион", 
                      count(E.id) as "Количество работодателей",
                      count(V.id) as "Количество вакансий"
                FROM public.areas as A
                LEFT JOIN public.employers as E on A.id=E.area
                FULL JOIN public.vacancies as V on E.area=V.area_id
                WHERE V.id is null
                GROUP BY A.name
                ORDER BY count(E.id) desc
                LIMIT 1
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
display(df)

,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3 = f'''SELECT 
                      E.name as "Наименование работодателя", 
                      count(distinct V.area_id) as "Количество регионов"
                FROM public.employers as E 
                LEFT JOIN public.vacancies as V on E.id=V.employer_id
                GROUP BY E.name
                ORDER BY count(V.area_id) desc
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
display(df.head())

,Наименование работодателя,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_5_4 = f'''SELECT
                      count(E.id) as "Количество работодателей"
                FROM public.employers as E 
                FULL JOIN public.employers_industries as EI on E.id=EI.employer_id
                WHERE EI.industry_id is null
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
display(df)

,Количество работодателей
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
query_5_5 = f'''SELECT
                      E.name as "Наименование работодателя"
                FROM public.employers as E 
                FULL JOIN public.employers_industries as EI on E.id=EI.employer_id
                GROUP BY E.name
                HAVING count(E.name) = 4
                ORDER BY "Наименование работодателя"
                OFFSET 2 LIMIT 1
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
display(df)

,Наименование работодателя
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
query_5_6 = f'''SELECT
                      I.name as "Сфера деятельности",
                      count(E.name) as "Количество работодателей"
                FROM public.employers as E 
                FULL JOIN public.employers_industries as EI on E.id=EI.employer_id
                FULL JOIN public.industries as I on I.id=EI.industry_id
                GROUP BY I.name
                HAVING I.name = 'Разработка программного обеспечения'
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
display(df)

,Сфера деятельности,Количество работодателей
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')

# поиск и выделение таблицы со списком городов-милионников
table = page.find_all("table")
table = table[1]                 

# создание и редактирование списка
cities_list = []                  
for element in table.find_all('td'):
  element = element.text
  cities_list.append(element)
cities_list = cities_list[1::5]

# исправление орфографической ошибки
cities_list[13] = 'Воронеж'        

print(cities_list)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград']


In [ ]:
# текст запроса
query_5_7 = f'''(SELECT 
                        A.name as "Город",
                        count(V.id) as "Количество вакансий" 
                 FROM public.vacancies as V
                 JOIN public.employers as E on E.id = V.employer_id
                 JOIN public.areas as A on A.id = V.area_id
                 WHERE E.name = 'Яндекс' and A.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 
                 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 
                 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
                 GROUP BY A.name)

                 UNION

                 (SELECT 
                         'Total',
                          count(v.id) as "Количество вакансий" 
                  FROM public.vacancies as V
                  JOIN public.employers as E on E.id = V.employer_id
                  JOIN public.areas as A on A.id = V.area_id
                  WHERE E.name = 'Яндекс' and A.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 
                  'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 
                  'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
                  )

                  ORDER BY "Количество вакансий"  asc
                  '''


In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
display(df)

,Город,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


<center>Выводы по анализу работодателей.</center>

В лидеры по количеству вакансий попали такие гиганты как "Сбер", "Яндекс", "Тинькофф", "Ростелеком" и "Газпромнефть". Эти компании работают более 20 лет и имеют многомиллиардные обороты. А  "Сбер", "Яндекс" и "Тинькофф" успешно развивают свои эко-системы в разных видах деятельности. Например, "Яндекс Такси", "Сбер Маркет", "Тинькофф Инвестиции".

 410 работодателей из региона "Россия" следует проверить на корректность и, возможно, исключить бы из выборки. Регистрация компании возможна только в конкретном городе(регионе). Если компания ведет деятельность и в разных регионах, она обязана открыть филиал/обособленное подразделение. Государственная регистрация связана со сдачей отчетности в определенные налоговые органы по месту нахождения и уплатой налогов в определенные уровни бюджета (например, 17% налога на прибыль уплачивается в региональный бюджет). Возможно, работодатель просто не до конца заполнил свой профиль на портале, что можно отнести к неполным данным. 

 Неуказание сферы деятельности (ровно как и несколько сфер деятельности несущественная ошибка, потому что одна компания может заниматься несколькими видами деятельности и не иметь основного (более 70% выручки от определенного вида деятельности). Это скорее неполные данные, восстановить которые можно через открытие ресурсы. Самый популярный - сайт nalog.ru.

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = f'''SELECT count(V.id) as "Количество вакансий"      
                FROM public.vacancies as V 
                WHERE V.name ilike '%data%' or V.name ilike '%данн%'
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
display(df)

,Количество вакансий
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
query_6_2 = f'''SELECT count(id) as "Количество вакансий для джуниоров"      
                FROM public.vacancies
                WHERE 
                      name not LIKE '%HTML%' 
                      
                      AND 
                     ((name ILIKE '%data scientist%') OR 
                      (name ILIKE '%data science%') OR 
                      (name ILIKE '%исследователь данных%') OR
                      (name LIKE '%ML%') OR
                      (name ILIKE '%machine learning%') OR
                      (name ILIKE '%машинн%обучен%'))

                      AND
                     (name ILIKE '%junior%'  OR
                      experience = 'Нет опыта' OR 
                      employment = 'Стажировка')   
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
display(df)

,Количество вакансий для джуниоров
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''SELECT count(*)  as "Количество вакансий со знанием SQL"
                FROM public.vacancies
                WHERE 
                      name not LIKE '%HTML%' 

                      AND 
                     ((name ILIKE '%data scientist%') OR 
                      (name ILIKE '%data science%') OR 
                      (name ILIKE '%исследователь данных%') OR
                      (name LIKE '%ML%') OR
                      (name ILIKE '%machine learning%') OR
                      (name ILIKE '%машинн%обучен%'))   

                     AND
                     (key_skills ILIKE '%SQL%' OR key_skills ILIKE '%postgres%')
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
display(df)

,Количество вакансий со знанием SQL
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''SELECT count(*) as "Количество вакансий со знанием Python "
                FROM public.vacancies
                WHERE 
                      name not LIKE '%HTML%' 

                      AND 
                     ((name ILIKE '%data scientist%') OR 
                      (name ILIKE '%data science%') OR 
                      (name ILIKE '%исследователь данных%') OR
                      (name LIKE '%ML%') OR
                      (name ILIKE '%machine learning%') OR
                      (name ILIKE '%машинн%обучен%'))   

                     AND
                     (key_skills ILIKE '%Python%')
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
display(df)

,Количество вакансий со знанием Pytho
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
query_6_5 = f'''SELECT  ROUND(AVG(LENGTH(key_skills)-LENGTH(REPLACE(key_skills, CHR(9), ''))+1), 2)
                        as "Среднее количество навыков"
                FROM public.vacancies
                WHERE 
                      name not LIKE '%HTML%' 

                      AND 
                     ((name ILIKE '%data scientist%') OR 
                      (name ILIKE '%data science%') OR 
                      (name ILIKE '%исследователь данных%') OR
                      (name LIKE '%ML%') OR
                      (name ILIKE '%machine learning%') OR
                      (name ILIKE '%машинн%обучен%'))   

                     AND
                     (key_skills IS NOT NULL)
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
display(df)

,Среднее количество навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса
query_6_6 = f'''SELECT 
                      count(*) as "Количество вакансий", 
                      experience as "Опыт работы", 
                      ROUND(AVG((coalesce(salary_from, salary_to)+coalesce(salary_to, salary_from))/2)) as "Средняя зарплата" 
                FROM public.vacancies
                WHERE 
                      name not LIKE '%HTML%' 

                      AND 
                     ((name ILIKE '%data scientist%') OR 
                      (name ILIKE '%data science%') OR 
                      (name ILIKE '%исследователь данных%') OR
                      (name LIKE '%ML%') OR
                      (name ILIKE '%machine learning%') OR
                      (name ILIKE '%машинн%обучен%'))   

                     AND
                     (salary_from IS NOT NULL OR salary_to IS NOT NULL)

                     GROUP BY experience
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
display(df)

,Количество вакансий,Опыт работы,Средняя зарплата
0,7,Нет опыта,74643.0
1,28,От 1 года до 3 лет,139675.0
2,27,От 3 до 6 лет,243115.0


<center>Выводы по предметному анализу.</center>

Ситуация по вакансиям для дата-саентистов примерно такая же, как и для всех остальных. Для новичков без опыта работы вакансий мало, но через год работы ситуация меняется. Однако формулировка "Нет опыта" бывает попадает в вакансии на поиск ТимЛида, поэтому нужно внимательно читать текст вакансии и требования. Отличие в том, что рынок труда по IT-профессиям пока только формируется, и бывает, что в вакансии "Аналитик данных" присутствуют требования о знании библиотек для машинного обучения, хотя это работа дата-саентиста. Поэтому, прежде всего для поиска работы нужно смотреть на требования и обязанности, а не на название вакансии. Я предполагаю, что именно с этим связано появление профессии IT-рекрутер, так как не все сотрудники кадровых служб знают отличия работы аналитика данных от работы дата-саентиста и дата-инженера.



# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

# выясним какие навыки ждут работодатели от начинающих специалистов
query_7 = f'''SELECT 
                      key_skills
                FROM public.vacancies
                WHERE 
                      name not LIKE '%HTML%' 

                      AND 
                     ((name ILIKE '%data scientist%') OR 
                      (name ILIKE '%data science%') OR 
                      (name ILIKE '%исследователь данных%') OR
                      (name LIKE '%ML%') OR
                      (name ILIKE '%machine learning%') OR
                      (name ILIKE '%машинн%обучен%'))

                      AND 
                      (experience = 'От 1 года до 3 лет' OR
                      experience = 'Нет опыта')
                      '''

In [ ]:
df = pd.read_sql_query(query_7, connection)
df = df.key_skills.apply(lambda x: str(x).split('\t'))

key_skills = collections.Counter()
for lists in df:
    for skill in lists:
        key_skills[skill] += 1
#выберем 10 самых распространенных навыков
display(key_skills.most_common(10)) 
# connection.close()

[('Python', 179),
 ('SQL', 134),
 ('Machine Learning', 54),
 ('Математическая статистика', 47),
 ('Математическое моделирование', 34),
 ('Git', 33),
 ('None', 31),
 ('Статистический анализ', 30),
 ('PyTorch', 28),
 ('Анализ данных', 25)]

In [ ]:
# узнаем, какие работодатели не указали регион работы/регистрации
query_8 = f'''SELECT 
                      E.name as "Наименование работодателя",
                      I.name as "Сфера деятельности"
              FROM public.employers as E
              JOIN public.areas as A on E.area=A.id
              JOIN public.employers_industries as EI on E.id=EI.employer_id
              FULL JOIN public.industries as I on I.id=EI.industry_id
                
              WHERE A.name LIKE 'Россия'
              GROUP BY E.name, I.name
              ORDER BY E.name
              '''

In [ ]:
df = pd.read_sql_query(query_8, connection)
display(df)

,Наименование работодателя,Сфера деятельности
0,1-й Советник,"Системная интеграция, автоматизации технологи..."
1,1С:ВДГБ,Разработка программного обеспечения
2,ALP GROUP,"Маркетинговые, рекламные, BTL, дизайнерские, E..."
3,ALP GROUP,Разработка программного обеспечения
4,ALP GROUP,"Системная интеграция, автоматизации технологи..."
...,...,...
891,ЮНИСТРИМ БАНК,"Финансово-кредитное посредничество (биржа, бро..."
892,ЮРИОН,"Промышленное, бытовое электрооборудование и эл..."
893,ЮРИОН,"Промышленное, бытовое электрооборудование и эл..."
894,ЮРИОН,"Электронно-вычислительная, оптическая, контрол..."


Итак, по данной выборке можно сделать следующие краткие выводы:


*   У нас имеется большая выборка с 49 тыс.вакансий от 23тыс.работодателей в более чем 1300 населенных пунктах (не только в России) и почти 300 видах деятельности.
*   Около 70% объявлений в базе - это вакансии на полный день и полную занятость. Еще 19% объявлений предлагают удаленную работу или гибкий график при полной занятости. Остальные вакансии в основном предполагают временную занятость.
*   Половина работадателей предпочитают не указывать заработную плату, и у него на это есть свои причины. 
*   Проще всего найти работу соискателю с опытом работы от 1 года до 3 лет в основном на рутинную работу.
*   Больше всего рабочих мест предлагают "Сбер", "Яндекс", "Тинькофф", "Ростелеком" и "Газпромнефть". 
*   Иметь несколько видов деятельности для компании нормально, если деятельность не указана - это некритично. Должны насторожить невнятные формулировки, например, "регион нахождения компании - Россия".
*   Вакансий для дата-саентистов не так много, но это скорее связано с тем, что профессия новая, и рынок только формируется. Для поиска лучше внимательно изучать текст вакансии и требования.
*   Самые востребованные навыки для начинающий дата-саентистов: Python, SQL, Machine Learning, Git, математическая статистика и математическое моделирование, статистический анализ и анализ данных, PyTorch.

